#Importamos las librerias que nos seran utiles

In [233]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import re

#Importamos los datos

In [234]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [235]:
sdf = pd.DataFrame(sharks)

### AGREGAR UN MINI EDA PARA MOSTRAR QUE HAY POCOS DATOS NUMERICOS, ETC

##Creo un nuevo data frame eliminando las columnas que no son relevantes para el analisis de la primer hipotesis ###DEFINIR FUNCION 

In [236]:
sdf.drop(columns=['pdf', 'href formula', 'href', 'Unnamed: 22', 'Unnamed: 23', 'Name', 'Investigator or Source', 'Sex ', 'Age', 'Date', 'Year', 'Country', 'Area', 'Location',
       'Activity', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', 'original order'], inplace=True)

#Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres ##DEFINIR FUNCION

In [237]:
compare = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [238]:
sdf['equal'] = compare

In [239]:
sdf['equal'].value_counts() 

False    19441
True      6282
Name: equal, dtype: int64

In [240]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal                0
dtype: int64

##SI COMPARAMOS LAS TRES COL ENTRE SI, EN DONDE MENOS DIF DE VAL HAY ES ENTRE CAS NBR Y CAS NBR.2, AUNQUE LA DI ES MINIMA. SIN EMBARGO DE LAS TRES COL LA QUE MENOS CANT DE NULL TIENE ES CASE NBR, NOS QUEDAMOS CON ELLA##DEJAMOS SOLO LAS COLUMNAS CASE NR Y TYPE

In [241]:
sdf.drop(columns=['Case Number.1', 'Case Number.2', 'equal'], inplace=True)
sdf.head()

,Case Number,Type
0,2018.06.25,Boating
1,2018.06.18,Unprovoked
2,2018.06.09,Invalid
3,2018.06.08,Unprovoked
4,2018.06.04,Provoked


##VAMOS A ELIMINAR LOS VALORES QUE NO SEAN FECHA, YA QUE NOS INTERESA VER LA ESTACIONALIDAD. Empezamos por los valores NaN, y luego añadiremos una columna solo con los valores que contengan informacion sobre el mes, ya que nos interesa solo la indfo mensual

In [242]:
sdf.dropna(inplace=True)

In [243]:
def filtracion(x):
    m = re.findall('\d{4}.\d{2}.\d{2}', x)
    if m and m[0][5:7] != '00':
        return m[0][5:7]

In [244]:
sdf['Month'] = sdf['Case Number'].apply(filtracion)

In [245]:
sdf.dropna()

,Case Number,Type,Month
0,2018.06.25,Boating,06
1,2018.06.18,Unprovoked,06
2,2018.06.09,Invalid,06
3,2018.06.08,Unprovoked,06
4,2018.06.04,Provoked,06
...,...,...,...
6155,1742.12.17,Unprovoked,12
6156,1738.04.06.R,Unprovoked,04
6159,1721.06.00,Unprovoked,06
6160,1703.03.26,Unprovoked,03


In [246]:
sns.barplot(x="Type", y="Month", data=sdf)

TypeError: Neither the `x` nor `y` variable appears to be numeric.